## Genius Setup

In [1]:
import sys
!{sys.executable} -m pip install lyricsgenius
!{sys.executable} -m pip install spotipy
!{sys.executable} -m pip install selenium
import pandas as pd
import requests
import lyricsgenius as genius
import spotipy
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

You are using pip version 9.0.1, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 9.0.1, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 9.0.1, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [10]:
genius_client_id = 'Qeii-UKMUUwsAYCuuemsF7CZk7R2bFDhjP1FMkqpozASvYsA4wQDlLmd2JFM3Vhm'
genius_secret_id = 'fWhV7VtwAL-5D32Ga9Y0Kyh_VFEo7XK0Jf8-3RMcZ5EoMXqsDpmFWFFPyobnibfMXJuJ-GZe69YEuXR-36VGAA'
genius_client_access_token = '1QVL91jZbgdOIhvYhqKku_zkFDf6MHEyLpXEITHU2tE2HGvAqYIPInTrQaWcInD7'
app_website_url = 'http://cis545genius.com'
redirect_uri = 'cis545genius.com'
base_url = 'https://api.genius.com'
driver_path = '/Users/arnabsarker/Downloads/chromedriver.exe'
# driver_path = "/mnt/c/Users/jeffr/Downloads/chromedriver_win32/chromedriver.exe"
artist_url = 'https://www.ranker.com/crowdranked-list/the-greatest-rappers-of-all-time'
page_downs = 0
api = genius.Genius(genius_client_access_token)

## Graph Generation

In [18]:
import pandas as pd

In [71]:
def get_samples_of_song(song_id):
    base_url = 'https://api.genius.com'
    search_url = base_url + '/songs/' + str(song_id)
    params = {}
    response = requests.get(search_url, params=params, headers=headers)
    json = response.json()
    samples = json['response']['song']['song_relationships'][0]['songs']
    sample_ids = [];
    for song in samples:
        sample_ids.append(song['id'])
    return sample_ids
    

In [144]:
def insert_samples_into_graph(song_id):
    #song_id = song_df[1]
    samples = get_samples_of_song(song_id)
    global graph_df
    global next_layer_df
    sample_dict_list = []
    for s in samples:
        sample_dict_list.append({'song': s})
    sample_df = pd.DataFrame(sample_dict_list)
    next_layer_df = next_layer_df.append(sample_df, ignore_index = True)
    sample_df = sample_df.rename(index=str, columns={"song": "sampled"})
    sample_df['sampler'] = song_id
    graph_df = graph_df.append(sample_df, ignore_index = True)
        

In [145]:
a = get_samples_of_song('51803')
print(a)

[81011, 349, 2797588]


In [156]:
# origin_map = [{'song': 51803}]
# origin_map = [{'song': 81011}, {'song': 349}, {'song': 2797588}]
#origin_df = pd.DataFrame(origin_map)

origin_df = pd.read_csv('toy_data_Jeffrey.csv')
origin_df = origin_df[['genius_id']].rename({'genius_id': 'song'})

In [157]:
headers = {'Authorization': 'Bearer %s' % genius_client_access_token}
depth = 4
graph_df = pd.DataFrame(columns=['sampler', 'sampled'])


curr_layer_df = origin_df

for i in range(0,depth):
    next_layer_df = pd.DataFrame(columns=['song'])
    curr_layer_df.applymap(insert_samples_into_graph)
    graph_df = graph_df.drop_duplicates()
    next_layer_df = next_layer_df.drop_duplicates()
    curr_layer_df = next_layer_df
    

In [158]:
graph_df

,sampled,sampler
0,2421991.0,2421955.0
1,1633008.0,2421955.0
0,1154377.0,2448.0
0,51899.0,482233.0
0,35672.0,86.0
0,947364.0,3404.0
0,260535.0,38.0
0,2082885.0,3468893.0
0,3113175.0,703738.0
0,3404932.0,500858.0


In [159]:
import sqlite3

engine = sqlite3.connect('Graph_DB')

graph_df.to_sql('graph_df', engine, if_exists='replace', index=True)